In [47]:
import hipscat
import healpy as hp
import pandas as pd
from tqdm import tqdm
from hipscat.inspection import plot_pixel_list
from hipscat.pixel_math import HealpixPixel
import numpy as np
import matplotlib.pyplot as plt

In [37]:
# plot_pixel_list([HealpixPixel(0,11), HealpixPixel(4, 78)])

In [38]:
step = 2

In [39]:
## i know this is dumb. i don't care.

def min_max_sep(bounds_a, bounds_b):
    min_sep = float("inf")
    max_sep = 0.0

    for i in range(4*step):
        for j in range (4*step):
            sep_sq = (bounds_a[0][i]-bounds_b[0][j])**2 + (bounds_a[1][i]-bounds_b[1][j])**2
            min_sep = min(min_sep, sep_sq)
            max_sep = max(max_sep, sep_sq)
    return (min_sep, max_sep)

In [40]:


gaia_full_partition_frame = pd.read_csv("gaia_partition_info.csv")
gaia_full_partition_list = [
            HealpixPixel(order, pixel)
            for order, pixel in zip(
                gaia_full_partition_frame["Norder"],
                gaia_full_partition_frame["Npix"],
            )
        ]
num_partitions = len(gaia_full_partition_list)
print(num_partitions, "partitions")
print(int(.5 * num_partitions * (num_partitions - 1)), "iterations")

3933 partitions
7732278 iterations


In [41]:

all_seps = []

for a in tqdm(range(0, num_partitions)):
    for b in range(a, num_partitions):

        bounds_a = hp.vec2dir(hp.boundaries(2**gaia_full_partition_list[a].order, gaia_full_partition_list[a].pixel, step=step, nest=True), lonlat=True)
        bounds_b = hp.vec2dir(hp.boundaries(2**gaia_full_partition_list[b].order, gaia_full_partition_list[b].pixel, step=step, nest=True), lonlat=True)
        all_seps.append(min_max_sep(bounds_a, bounds_b))

100%|██████████| 3933/3933 [09:51<00:00,  6.64it/s] 


In [42]:
len(all_seps)

7736211

In [43]:
all_seps[:10]

[(0.0, 506.24999999999966),
 (0.0, 1236.6206396835553),
 (6299.120639683556, 15411.620639683555),
 (4972.652051530026, 13072.652051530025),
 (8516.402051530025, 18641.402051530025),
 (11289.449049432578, 23391.34255485142),
 (7239.44904943258, 16351.949049432578),
 (10791.34255485142, 24620.924450944753),
 (6066.342554851419, 15516.34255485142),
 (9433.42445094475, 26107.389504989897)]

In [46]:
min_map_arrays = np.array(all_seps).T
min_map_arrays[0:10]
mins = min_map_arrays[0]
maxs = min_map_arrays[1]

In [48]:
pix_a_order = []
pix_a_pixel = []
pix_b_order = []
pix_b_pixel = []

for a in tqdm(range(0, num_partitions)):
    for b in range(a, num_partitions):
        pix_a_order.append(gaia_full_partition_list[a].order)
        pix_a_pixel.append(gaia_full_partition_list[a].pixel)
        pix_b_order.append(gaia_full_partition_list[b].order)
        pix_b_pixel.append(gaia_full_partition_list[b].pixel)


big_beautiful_frame = pd.DataFrame({"Norder_a": pix_a_order,
                                    "Npix_a": pix_a_pixel,
                                    "Norder_b": pix_b_order,
                                    "Npix_b": pix_b_pixel,
                                    "min_sep": mins,
                                    "max_sep": maxs,
                                    })
big_beautiful_frame.to_csv("bbf.csv")

100%|██████████| 3933/3933 [00:01<00:00, 2135.54it/s]
